In [34]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
tik_tok_df = pd.read_csv('tiktok_app_reviews.csv')

/Users/alexstake/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
tik_tok_df =tik_tok_df.drop(columns='review_id')

In [5]:
tik_tok_df.head()

,review_text,upvotes,app_version,posted_date
0,It is an excellent solution to finding videos ...,402,21.2.5,2021-09-20 07:50:23
1,Great app works really well so many features s...,67,21.2.5,2021-09-19 23:10:47
2,I down loaded app but didnt work . i love this...,24,21.2.5,2021-09-19 08:59:34
3,IT A GREAT APP AND FUN TO USE! HOWEVER ARBITRA...,18,21.2.5,2021-09-19 16:34:43
4,Currently it's working good for me except a fe...,232,21.1.41,2021-09-14 06:35:02


In [25]:
tik_tok_df['date'] = pd.to_datetime(tik_tok_df['posted_date'], infer_datetime_format= True).dt.date

In [35]:
split_date= dt.date(2020,1,1)

In [41]:
tik_tok_20_21 = tik_tok_df.loc[tik_tok_df['date']>split_date]

In [42]:
tik_tok_20_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1937876 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 88.7+ MB


In [43]:
tik_tok_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3646476 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 139.1+ MB


In [50]:
tik_tok_20_21.review_text.isnull().sum()

16

Only 16 null reviews out of 1.9 million, so can probably drop them without too much impact

In [55]:
tik_tok_20_21.loc[tik_tok_20_21.review_text.isnull()]

,review_text,upvotes,app_version,posted_date,date
1713596,NaN,0,19.5.43,2021-06-16 11:39:27,2021-06-16
1724879,NaN,0,19.5.4,2021-05-19 03:56:05,2021-05-19
1729379,NaN,0,19.3.4,2021-05-25 23:51:17,2021-05-25
1762648,NaN,0,18.8.41,2021-04-09 05:12:47,2021-04-09
1782371,NaN,0,18.6.6,2021-03-04 19:01:34,2021-03-04
1824753,NaN,0,18.3.6,2021-01-31 07:27:14,2021-01-31
2079417,NaN,0,15.9.6,2020-05-11 14:45:00,2020-05-11
2117121,NaN,0,15.7.5,2020-04-20 08:48:29,2020-04-20
2144000,NaN,0,15.5.42,2020-05-04 01:45:30,2020-05-04
2212880,NaN,0,15.2.4,2020-03-15 04:03:22,2020-03-15


In [58]:
no_null_reviews_df = tik_tok_20_21.dropna(subset=['review_text'])

In [59]:
no_null_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1937860 entries, 0 to 3646475
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   review_text  object
 1   upvotes      int64 
 2   app_version  object
 3   posted_date  object
 4   date         object
dtypes: int64(1), object(4)
memory usage: 88.7+ MB
